### Package requirements
    * conda install plotly
    * conda install -c conda-forge rdkit

### About 
Cell below contains a function that makes a scatter plot and provides a molecular graph on hover action. The inputs require a dataframe, and the names of the x, y, and smiles columns in that dataframe
    

In [3]:
import io

import plotly.graph_objs as go
from ipywidgets import HTML, Image, Layout, HBox, VBox

from rdkit import Chem
from rdkit.Chem.Draw import MolToImage

def scatter_mol_hover(dataframe, x_col_name, y_col_name, smiles_col_name):
    """

    :param dataframe: input pandas dataframe
    :param x_col_name: x-axis column name
    :param y_col_name: y-axis column name
    :param smiles_col_name: smiles column name
        """

    # helper function that converts smiles to byte array image
    def smi_to_bytes_img(smi):
        img = MolToImage(Chem.MolFromSmiles(smi), size=(300, 300))
        io_bytes = io.BytesIO()
        img.save(io_bytes, format='PNG')           
        return io_bytes.getvalue()

    # hover action function
    def hover_fn(trace, points, state):
        ind = points.point_inds[0]
        smi = dataframe[smiles_col_name].iloc[ind]
        details.value = smi         
        image_widget.value = smi_to_bytes_img(smi)


    # scatter plot
    fig = go.FigureWidget(
        data=[
            dict(
                type='scattergl',
                x=dataframe[x_col_name],
                y=dataframe[y_col_name],
                mode='markers',
                #marker_size=dataframe['yield']
                #marker_color=dataframe['type'].astype('category').cat.codes
            )
        ],
        layout = go.Layout(
        autosize=False,
        template="plotly_white",
        width=600,
        height=600,
        xaxis_title=x_col_name,
        yaxis_title=y_col_name
        )
    )

    # initialize details/image widgets
    smi = dataframe[smiles_col_name].iloc[0]
    # details space
    details = HTML(value=smi)
    # image widget
    image_widget = Image(value=smi_to_bytes_img(smi))
   

    # assign hover action to the scatter plot
    scatter = fig.data[0]
    scatter.on_hover(hover_fn)

    return HBox([fig, VBox([image_widget, details])])

In [4]:
import pandas as pd

# Tiny example
df = pd.DataFrame([[0.5,0.5, '1a', 'BrCc1ccc(cc1)B1OC(C(O1)(C)C)(C)C', 10],
                   [0,1, '1a', 'COc1cccc(c1B(O)O)Cl', 20],
                   [1,0, '2b', 'OB(c1ccc(cc1C(F)(F)F)C(F)(F)F)O', 30]],
                  columns=['x','y', 'type','label', 'yield'])

x_col_name = 'x'
y_col_name = 'y'
smi_col_name = 'label'

scatter_mol_hover(df, 'x', 'y', 'label')

    'data': [{'marker': {'size': array([10, 20, 30], dtype=int64)},
            …

In [3]:
# Larger example
#df = pd.read_csv("../../../Projects/Novartis/data/Boronic_reagents_dft.csv")
#scatter_mol_hover(df, 'global_molar_mass', 'B_VBur', 'can')